In [3]:
from pickle import load, dump
from os import makedirs
from os.path import dirname

import pandas as pd

In [4]:
   
def predict_data(data, model, ml_model: str, lags, **kwargs):
    if ml_model[0: 3] == 'svr' or ml_model[0: 3] == 'mlp' or ml_model[0:2] == 'rf':
        data = data.testing_set
        return model.predict(data[:, lags])
    elif ml_model[0: 7] == 'xgboost':
        data = data.testing_set

        return model.predict(data[:, lags])
    elif ml_model[0: 4] == 'lstm':
        data = data.testing_set
        data = data[:, lags]
        data = data.reshape((data.shape[0], data.shape[1], 1))
        return model.predict(data).ravel()
    elif ml_model[0: 3] in ['dee', 'tft']:
        data = data.testing_x
        pred = []

        for d in data:
            predictions = model.predict(d)

            for x in predictions:
                pred.append(x.mean[0])

        return pred

    elif ml_model[0: 6] == 'da-rnn':
        data = data.testing_x
        import torch

        with torch.no_grad():
            a = model(data.cuda())

        return list(a.cpu().repeat(1, 2).numpy()[:, -1])

    elif ml_model[0: 5] == 'arima':
        data = data.testing_set
        arima_forecast = kwargs.get('arima_forecast')
        if arima_forecast:
            if arima_forecast == 'in_sample':
                return model.predict_in_sample(data)[data]
            elif arima_forecast == 'out_sample':
                return model.predict(data + 1)[-1]
        else:
            return model.predict(len(data))

In [29]:
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import numpy as np
import io
summary_str = io.StringIO()

lags = [0, 40, 30, 20, 10, 50, 30, 60, 10, 30, 30]
modelos = ['arima', 'da-rnn', 'deep-ar', 'deep-state', 'lstm', 'tft', 'mlp', 'rf', 'svr', 'xgboost']

modelos = ['svr']

for metrica in ['response_time']:
    for index in range(1, 11):
        parameters = []
        summary_str = io.StringIO()
        
        for modelo in modelos:            
            #for lag in lags:
            lag = lags[index]
            
            # Open the pickle file and load the model manually using torch.load
            with open('pickle/microservice '+str(index)+'/'+metrica+'/hyper_parameter/'+modelo+str(lag)+'.pkl', 'rb') as f:
                model = load(f)  # Load the object
            
            # Now manually load the PyTorch model onto the CPU
            if isinstance(model, torch.nn.Module):
                model = torch.load(f, map_location=torch.device('cpu'))
            
            model = model['model']
            
            if modelo == "lstm":
                model.summary(print_fn=lambda x: summary_str.write(x + "\n"))
                summary_str_value = summary_str.getvalue()
                parameters.append(summary_str_value)
                #parameters.append(print(model.to_json()))
            elif modelo == "xgboost":
                parameters.append(model.__dict__) 
            elif modelo == "deep-state":
                parameters.append(model.__dict__)
            elif modelo == "deep-ar":
                parameters.append(model.__dict__)
            elif modelo == "da-rnn":
                parameters.append(model.__dict__)
            elif modelo == "tft":
                parameters.append(model.__dict__)
            else:
                parameters.append(model)
                print(model)
                    
                #print(predictions[modelo][lag])
        
        #dump(df, open('forecasting/microservice '+str(index)+'/'+metrica+'.pkl', 'wb'))
    result_df = pd.DataFrame(data=np.array(parameters).reshape(-1, len(lags)), index=modelos, columns=lags)
    print(result_df)
        #print(result_df)
        #result_df.to_csv('hyper_parameter.csv')
        #result_df.to_csv('other-results/hyperparameters/microservice '+str(index)+'/'+metrica+'.csv')

SVR(C=10, epsilon=0.0001, gamma=0.01)
SVR(C=0.1, gamma=1)
SVR(C=10000, epsilon=0.0001, gamma=0.001)
SVR(C=1000, epsilon=0.001, gamma=0.1)
SVR(C=10, gamma=0.01, kernel='sigmoid')
SVR(C=0.1, gamma=0.001, kernel='sigmoid')
SVR(C=10, epsilon=0.0001, gamma=0.01, kernel='sigmoid')
SVR(C=10000, epsilon=0.001, gamma=0.001)
SVR(C=1000, epsilon=0.001, gamma=0.001, kernel='sigmoid')
SVR(C=1000, gamma=0.01)


ValueError: cannot reshape array of size 1 into shape (11)

In [30]:
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import numpy as np
import io
summary_str = io.StringIO()

lags = [10, 20, 30, 40, 50, 60]
modelos = ['arima', 'da-rnn', 'deep-ar', 'deep-state', 'lstm', 'tft', 'mlp', 'rf', 'svr', 'xgboost']

#modelos = ['svr']

for metrica in ['cpu']:
    for index in range(1, 11):
        parameters = []
        summary_str = io.StringIO()
        
        for modelo in modelos:            
            for lag in lags:
                # Open the pickle file and load the model manually using torch.load
                with open('pickle/microservice '+str(index)+'/'+metrica+'/hyper_parameter/'+modelo+str(lag)+'.pkl', 'rb') as f:
                    model = load(f)  # Load the object
                
                # Now manually load the PyTorch model onto the CPU
                if isinstance(model, torch.nn.Module):
                    model = torch.load(f, map_location=torch.device('cpu'))
                
                model = model['model']
                
                if modelo == "lstm":
                    model.summary(print_fn=lambda x: summary_str.write(x + "\n"))
                    summary_str_value = summary_str.getvalue()
                    parameters.append(summary_str_value)
                    #parameters.append(print(model.to_json()))
                elif modelo == "xgboost":
                    parameters.append(model.__dict__) 
                elif modelo == "deep-state":
                    parameters.append(model.__dict__)
                elif modelo == "deep-ar":
                    parameters.append(model.__dict__)
                elif modelo == "da-rnn":
                    parameters.append(model.__dict__)
                elif modelo == "tft":
                    parameters.append(model.__dict__)
                else:
                    parameters.append(model)
                    
                #print(predictions[modelo][lag])
        
        #dump(df, open('forecasting/microservice '+str(index)+'/'+metrica+'.pkl', 'wb'))
        result_df = pd.DataFrame(data=np.array(parameters).reshape(-1, len(lags)), index=modelos, columns=lags)
        #print(result_df)
        #result_df.to_csv('hyper_parameter.csv')
        result_df.to_csv('other-results/hyperparameters/microservice '+str(index)+'/'+metrica+'.csv')

/home/wellison/Development/PycharmProjects/comparative-analysis-microservices-prediction/venv/lib/python3.8/site-packages/pmdarima/arima/arima.py:854: UserWarning: You've deserialized an ARIMA from a version (2.0.1) that does not match your installed version of pmdarima (1.8.5). This could cause unforeseen behavior.
  warnings.warn("You've deserialized an ARIMA from a version (%s) "
[15:15:30] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of MXNet v10600. May cause undefined behavior. Please update MXNet if you encounter any issue
[15:15:30] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of MXNet v10600. May cause undefined behavior. Please update MXNet if you encounter any issue
[15:15:30] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of MXNet v10600. May cause undefined behavior. Please update MXNet if you e